<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/keras/keras_Titanic_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Verifica se a GPU está ativa

In [1]:
!nvidia-smi

Wed Feb 20 00:15:44 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Construindo uma Rede Neural - Titanic

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

In [0]:
#!python -m pip install --upgrade https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.12.0-py3-none-any.whl
#!pip install keras

In [0]:
np.random.seed(1337)

In [4]:
# leitura dos dados
#from google.colab import files
#uploaded = files.upload()
!wget https://github.com/vladimiralencar/DeepLearning-LANA/raw/master/keras/titanic.csv
!wget https://github.com/vladimiralencar/DeepLearning-LANA/raw/master/keras/titanic_test.csv
!wget https://github.com/vladimiralencar/DeepLearning-LANA/raw/master/keras/titanic_train.csv
!wget https://github.com/vladimiralencar/DeepLearning-LANA/raw/master/keras/titanic_gender_submission.csv



--2019-02-20 09:07:56--  https://github.com/vladimiralencar/DeepLearning-LANA/raw/master/keras/titanic.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/keras/titanic.csv [following]
--2019-02-20 09:07:57--  https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/keras/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16816 (16K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  16.42K  --.-KB/s    in 0.01s   

2019-02-20 09:07:57 (1.35 MB/s) - ‘titanic.csv’ saved [16816/16

In [5]:
!ls

sample_data  titanic_gender_submission.csv  titanic_train.csv
titanic.csv  titanic_test.csv


In [0]:
#import io
#df2 = pd.read_csv(io.StringIO(uploaded['Filename.csv']))

In [0]:
# folder_titan = '/media/datasets/DeepLearningI/Cap02/titanic.csv'
df = pd.read_csv('titanic.csv')

In [8]:
df.head()

,Survived,Class,Sex,Age,Fare
0,0,3,1,22.0,7.2500
1,1,1,0,38.0,71.2833
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500


In [9]:
len(df)

891

In [10]:
df_train = df.iloc[:712, :]

df_train.head()

,Survived,Class,Sex,Age,Fare
0,0,3,1,22.0,7.2500
1,1,1,0,38.0,71.2833
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500


In [0]:
scaler = StandardScaler()
features = ['Class', 'Sex', 'Age', 'Fare']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [12]:
X_train[:5]

array([[ 0.83290956,  0.74926865, -0.61259594, -0.51933199],
       [-1.55353553, -1.33463478,  0.6184268 ,  0.79718222],
       [ 0.83290956, -1.33463478, -0.30484025, -0.5054541 ],
       [-1.55353553, -1.33463478,  0.38761004,  0.42333654],
       [ 0.83290956,  0.74926865,  0.38761004, -0.50288412]])

In [13]:
y_train_onehot[:3]

array([[1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [0]:
df_test = df.iloc[712:, :]

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

In [15]:
np.set_printoptions(precision=5, suppress=True)
X_train[:5]

array([[ 0.83291,  0.74927, -0.6126 , -0.51933],
       [-1.55354, -1.33463,  0.61843,  0.79718],
       [ 0.83291, -1.33463, -0.30484, -0.50545],
       [-1.55354, -1.33463,  0.38761,  0.42334],
       [ 0.83291,  0.74927,  0.38761, -0.50288]])

In [16]:
y_train[:5]

array([0, 1, 1, 1, 0])

## Criando um Modelo de Árvore de Decisão ( Benchmark)

In [17]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['Survived'].values)

y_prediction = model.predict(X_test)
print ("\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

Acurácia 0.8100558659217877


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## Rede Neural com 1 Camada

In [18]:
print(features)
X_train[:3]

['Class', 'Sex', 'Age', 'Fare']


array([[ 0.83291,  0.74927, -0.6126 , -0.51933],
       [-1.55354, -1.33463,  0.61843,  0.79718],
       [ 0.83291, -1.33463, -0.30484, -0.50545]])

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()

model.add(Dense(input_dim=4, units=2)) # saída - units

model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\nTempo gasto: %s segundos' % str(time() - start))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
712/712 [==============================] - 3s 4ms/step - loss: 1.4410 - acc: 0.3413

Tempo gasto: 2.864208459854126 segundos


In [20]:
y_prediction = model.predict_classes(X_test)
print ("\n\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))



Acurácia 0.3240223463687151


## Rede Neural com 2 Camadas

In [21]:
start = time()

model = Sequential()

model.add(Dense(input_dim=4, units = 100)) # units - numero de neuronios

model.add(Dense(units=2))

model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\nTempo gasto: %s segundos' % str(time() - start))

Epoch 1/1
712/712 [==============================] - 0s 287us/step - loss: 0.6605 - acc: 0.6517

Tempo gasto: 0.47099947929382324 segundos


In [22]:
y_prediction = model.predict_classes(X_test)
print ("\n\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))



Acurácia 0.8100558659217877


## Rede Neural com 3 Camadas

In [23]:
start = time()

model = Sequential()

model.add(Dense(input_dim=4, units=100))

model.add(Dense(units=100))

model.add(Dense(units=2))

model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print ('\nTempo gasto: %s segundos' % str(time() - start))

Epoch 1/1
712/712 [==============================] - 0s 537us/step - loss: 0.7281 - acc: 0.5239

Tempo gasto: 0.6814820766448975 segundos


In [24]:
y_prediction = model.predict_classes(X_test)
print ("\n\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))



Acurácia 0.776536312849162
